In [ ]:
import json
from collections import defaultdict

def calculate_move_frequencies(input_file, output_file):
    """
    根據每場比賽提取棋步序列，並計算每一步棋的頻率。
    """
    # 初始化數據結構
    move_frequencies = defaultdict(int)
    total_moves = 0

    # 讀取比賽數據
    with open(input_file, "r") as file:
        games = json.load(file)

    # 遍歷每場比賽
    for game_idx, game in enumerate(games):
        moves = game.get("moves", "")
        if not moves:
            print(f"比賽 {game_idx} 沒有棋譜，跳過...")
            continue

        # 分割棋步序列
        move_list = [move for move in moves.split() if not move.endswith(".")]

        # 計算每一步棋的頻率
        for move in move_list:
            move_frequencies[move] += 1
            total_moves += 1

    # 計算概率
    move_probabilities = []
    for move, count in move_frequencies.items():
        probability = (count / total_moves) * 100
        move_probabilities.append({
            "Move": move,
            "Count": count,
            "Probability%": f"{probability:.2f}%"
        })

    # 按頻率排序
    move_probabilities.sort(key=lambda x: x["Count"], reverse=True)

    # 保存結果
    with open(output_file, "w") as file:
        json.dump(move_probabilities, file, indent=4)

    print(f"棋步頻率已保存至 {output_file}")

# 執行
input_file = "./game_data_final.json"  # 你的比賽數據文件
output_file = "./move_frequencies.json"  # 保存的輸出文件
calculate_move_frequencies(input_file, output_file)

In [23]:
import json
from collections import defaultdict

def calculate_sequence_recommendations(input_file, output_file):
    """
    基於棋步序列統計推薦數據，記錄每個棋步序列對應的下一步及其概率。
    """
    # 初始化數據結構
    sequence_counts = defaultdict(lambda: defaultdict(int))  # 每個序列對應的下一步棋及次數
    sequence_total_counts = defaultdict(int)  # 每個序列的總次數

    # 讀取比賽數據
    with open(input_file, "r") as file:
        games = json.load(file)

    # 遍歷每場比賽
    for game_idx, game in enumerate(games):
        moves = game.get("moves", "")
        if not moves:
            print(f"比賽 {game_idx} 沒有棋譜，跳過...")
            continue

        # 分割棋步，移除步數標記（如 1.、2. 等）
        move_list = [move for move in moves.split() if not move.endswith(".")]

        # 遍歷棋步，記錄每個序列的下一步
        for i in range(len(move_list) - 1):  # 遍歷到倒數第二步
            sequence = " ".join(move_list[:i+1])  # 當前的棋步序列
            next_move = move_list[i+1]  # 下一步棋
            sequence_counts[sequence][next_move] += 1  # 記錄下一步的次數
            sequence_total_counts[sequence] += 1  # 更新序列的總次數

    # 構建推薦數據
    recommendations = {}
    for sequence, moves in sequence_counts.items():
        move_data = []
        for move, count in moves.items():
            probability = (count / sequence_total_counts[sequence]) * 100  # 計算概率
            move_data.append({
                "Move": move,
                "Count": count,
                "Probability%": f"{probability:.2f}%"
            })
        move_data.sort(key=lambda x: -x["Count"])  # 按次數排序
        recommendations[sequence] = move_data

    # 保存推薦數據
    with open(output_file, "w") as file:
        json.dump(recommendations, file, indent=4)

    print(f"推薦數據已保存至 {output_file}")

# 執行程式
input_file = "./game_data_final.json"  # 比賽數據文件
output_file = "./sequence_recommendations.json"  # 輸出推薦數據文件
calculate_sequence_recommendations(input_file, output_file)

推薦數據已保存至 ./sequence_recommendations.json
